In [60]:
import pandas as pd
chatbot_data = pd.read_excel("word_rag.xlsx")

chatbot_data.info()
display(chatbot_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rule      306 non-null    object
 1   response  306 non-null    object
dtypes: object(2)
memory usage: 4.9+ KB


,rule,response
0,ERA,"Earned Run Average의 약자로, 투수가 9이닝 동안 허용한 자책점의 평..."
1,1군 명단,프로 야구 팀의 가장 핵심적인 선수단을 의미합니다. 주로 1군 엔트리에 등록되어 정...
2,변화구,"날아가다가 궤도가 바뀌는 구종을 묶어서 말합니다. 커브볼, 슬라이더, 너클볼, 체인..."
3,배터리,투수와 포수를 묶어서 말할 때 쓰는 단어입니다.
4,자책점,"투수의 책임이 되는 실점을 의미합니다. 즉, 투수의 투구로 인해 발생한 실점 중에서..."
...,...,...
301,주루,주자가 베이스 사이로 뛰는 것을 주루라고 합니다.
302,진루,다음 베이스로 이동하는 것을 진루라고 합니다.
303,귀루,원래 베이스로 돌아가는 것을 귀루라고 합니다.
304,타석,타자가 공을 치기 위해 서는 구역을 말합니다. 타석에 선 타자는 투수가 투구를 시작...


In [61]:
# 딕셔너리 형태로 변환
baseball_dict = dict(zip(chatbot_data['rule'], chatbot_data['response']))
print(len(baseball_dict))

306


In [62]:
# 불용어
stop_words = ['이', '가', '은', '는', '을', '를', '이야', '야', '뭐야', '은', '는', '에', '에서', '에게', '의', '와', '과', '뭐지', '이름은', '뜻은', '무엇', '의미', '뭐', '좀', '설명', '해주']

def preprocess_request(request):
    request = request.lower()
    request = request.replace(" ", "")
    processed_text = request
    for word in stop_words:
        # '뭐야'를 '뭐야?'로 질문할 수도 있으므로 단어 끝에 있는 것을 제거
        processed_text = processed_text.replace(word, "")

    return processed_text

In [63]:
def chat(request, baseball_dict):
    # 1. 띄어쓰기만 제거한 원본 요청 (정확도 향상을 위해)
    request_no_space = request.replace(" ", "")
    
    # 2. 전처리된 요청 (조사 및 의문사 제거)
    processed_request = preprocess_request(request)

    # 3. 딕셔너리의 키를 순회하며 비교
    for term, response in baseball_dict.items():
        # 룰(Key)도 띄어쓰기 제거
        term_no_space = term.replace(" ", "")
        
        # 4. 일치 조건 검사
        # Case A: 띄어쓰기 제거된 사용자의 질문이, 띄어쓰기 제거된 '용어'를 포함하는가?
        # (예: "삼진이뭐야"가 "삼진"을 포함)
        if term_no_space in request_no_space:
            return response
            
        # Case B: 불필요 단어를 제거한 질문이, 띄어쓰기 제거된 '용어'와 동일한가?
        # (예: "삼진이뭐야" -> "삼진"이 "삼진"과 동일)
        if processed_request == term_no_space:
             return response
            
    return "죄송해요. 아직 그 용어는 준비되지 않았어요. 다른 질문도 해보실래요?"

In [ ]:
# 챗봇
while True:
    req = input('모르는 용어를 질문하세요! 질문을 종료하고 싶으면 끝이라고 입력하세요.')
    if req == '끝':
        break
    else:
        print('USER : ', req)
        print('BAIS : ', chat(req, baseball_dict))
        print('=======================================================')

USER :  플라이
BAIS :  뜬공과 유사하게, 공이 높게 떠서 외야수나 내야수가 잡을 수 있는 타구를 의미합니다.
USER :  볼넷
BAIS :  스트라이크존을 벗어난 투구에 대해 타자가 스윙하지 않았을 때 심판이 선언하는 판정입니다. 볼이 4개가 되면 타자는 볼넷으로 1루를 부여받습니다.
USER :  뭐야
BAIS :  죄송해요. 아직 그 용어는 준비되지 않았어요. 다른 질문도 해보실래요?
USER :  타자는 뭐
BAIS :  공격시에 배트를 가지고 타석에서 공을 치는 선수를 말합니다. 
USER :  도루
BAIS :  투수가 투구 동작을 하는 사이에 주자가 다음 베이스를 미리 뛰어 진루를 시도하는 플레이입니다. 성공하면 도루가 기록됩니다.
USER :  와
BAIS :  죄송해요. 아직 그 용어는 준비되지 않았어요. 다른 질문도 해보실래요?


---

# 여기서는 그냥 규칙 물어보는 거 키워드로 테스트 한 번 해본 거예요~

In [42]:
import pandas as pd
df = pd.read_excel("question_rag.xlsx")

display(df)

,질문,키워드,정답(설명)
0,"타자가 공을 맞히지도 못했는데, 심판이 갑자기 손을 번쩍 들면서 아웃이라고 하더라....","타자, 심판, 아웃, 공, 맞지도","타자가 공을 맞히지 못해도, 스트라이크로 판정된 공이 세 번 쌓이면 타자는 아웃이 ..."
1,배트에 살짝 스친 공이 뒤로 관중석 쪽으로 날아갔는데도 같은 타자가 계속 다시 치더...,"배트, 스친, 관중석, 계속",타자가 친 공이 뒤로 가거나 옆으로 빠져서 파울 구역으로 나가면 대부분의 경우 '파...
2,"TV 화면에 S랑 B 옆에 숫자가 계속 바뀌던데, 저 숫자는 뭘 뜻하는 거야? 어떤...","S, B, 숫자","화면의 S는 스트라이크, B는 볼 개수를 뜻한다. 스트라이크가 세 개가 되면 타자는..."
3,타자가 공을 못 치고 가만히 서만 있었는데 갑자기 1루 쪽으로 천천히 걸어 나가더라...,"타자, 공, 가만히, 걸어, 안",투수가 던진 공이 스트라이크존을 벗어났는데 타자가 휘두르지 않으면 '볼'로 센다. ...
4,"타자가 공에 몸을 맞은 것 같은데도 심판이 그냥 계속하라고 하던데, 어떤 때는 맞으...","공, 몸, 계속","투수가 던진 공에 타자의 몸이나 장비가 맞으면 보통 타자를 1루로 보내지만, 항상 ..."
5,"타자가 안타를 안 쳤는데 1루에 있던 주자가 갑자기 2루로 뛰다가 잡히기도 하던데,...","안타, 1루, 2루, 주자, 뛰다, 뛰어",투수가 공을 던질 준비를 하고 있을 때 주자는 다음 베이스로 '몰래' 뛰어가 볼 수...
6,"타자가 공을 높이 쳐서 잡혔는데, 3루에 있던 사람은 공이 잡힌 뒤에야 집으로 뛰어...","높이, 잡혀, 뛰어",뜬공이 떠 있는 동안 주자는 원래 있던 베이스를 떠나지 말아야 한다. 수비가 공을 ...
7,"타자가 공을 땅에 굴렸는데, 수비수가 공을 잡고 베이스만 밟았는데도 1루로 가던 사...","공, 땅, 바로, 아웃, 베이스","타자가 공을 치면 무조건 1루로 가야 해서, 1루에 있던 주자는 2루로 밀려날 의무..."
8,"포수 뒤로 공이 멀리 빠지니까, 3루에 있던 사람이 바로 집으로 뛰어 들어와서 점수...","포수, 공, 멀리, 빠져","투수가 던진 공이 너무 빗나가서 포수가 정상적으로 잡기 힘든 정도로 뒤로 빠지면, ..."
9,"투수가 공을 던지려다 말고 갑자기 멈춰 서니까, 심판이 손을 들고 주자들을 한 칸씩...","투수, 공, 던지려다, 멈춰","주자가 있을 때 투수는 정해진 방식대로만 투구 동작을 해야 하고, 주자를 속이려는 ..."


In [43]:
df['keyword'] = df['키워드'].apply(lambda x: set(item.strip() for item in x.split(',')))
print(df['keyword'].head(5))

0    {심판, 공, 타자, 아웃, 맞지도}
1       {계속, 스친, 배트, 관중석}
2              {B, 숫자, S}
3     {걸어, 가만히, 공, 타자, 안}
4              {계속, 공, 몸}
Name: keyword, dtype: object


In [50]:
# 불용어
stop_words = ['이', '가', '은', '는', '을', '를', '이야', '야', '뭐야', '은', '는', '에', '에서', '에게', '의', '와', '과', '뭐지', '이름은', '뜻은', '무엇', '의미', '뭐', '좀', '설명', '해주']

def preprocess_request(request):
    request = request.replace(" ", "")
    for word in stop_words:
        request = request.replace(word, "")
    return set(request)

In [51]:
def baseball_rule_chatbot(data, threshold=2):
    user_question = input("야구 규칙에 대해 궁금한 점을 질문해주세요!")
    print(f"입력된 질문 : {user_question}")

    user_keywords = preprocess_request(user_question)
    print(f"추출된 키워드 : {list(user_keywords)}")

    best_score = 0
    best_answer = "죄송합니다. 질문의 의도를 정확히 파악하지 못했습니다. 질문에 야구 용어(키워드)를 포함하여 다시 질문해 주시겠어요?"

    for _, row in data.iterrows():
        score = len(user_keywords.intersection(row['keyword']))
        if score > best_score:
            best_score = score
            best_answer = row['정답(설명)']

    print(f"\n최대 일치 점수 : {best_score}")
    print("-------챗봇 답변------")
    if best_score >= threshold:
        print("가장 유사한 질문에 대한 설명입니다.")
        print(best_answer)
    else:
        print(best_answer)

In [52]:
baseball_rule_chatbot(df)

입력된 질문 : TV에서 S랑 B랑 계속 바뀌던데 뭔 의미야?
추출된 키워드 : ['B', '뀌', '던', '랑', '바', '미', '?', '서', '계', '데', 'S', 'V', '속', 'T', '뭔']

최대 일치 점수 : 2
-------챗봇 답변------
가장 유사한 질문에 대한 설명입니다.
화면의 S는 스트라이크, B는 볼 개수를 뜻한다. 스트라이크가 세 개가 되면 타자는 아웃이 되고, 볼이 네 개가 되면 타자는 자동으로 1루로 나갈 수 있다. 그래서 스트라이크가 두 개, 볼이 세 개인 상황처럼 다음 한 공으로 아웃이 될 수도, 1루를 얻을 수도 있는 순간에는 선수와 관중 모두 긴장감이 크게 올라가는 것이다.
